# asyncio — Asynchronous I/O

asyncio is a library to write concurrent code using the async/await syntax.

High-level APIs
- Coroutines and Tasks
- Streams
- Synchronization Primitives
- Subprocesses
- Queues
- Exceptions

Low-level APIs
- Event Loop
- Futures
- Transports and Protocols
- Policies
- Platform Support

## Coroutines and Tasks¶

### Coroutines¶

Coroutines declared with the async/await syntax is the preferred way of writing asyncio applications. For example, the following snippet of code (requires Python 3.7+) prints “hello”, waits 1 second, and then prints “world”:

In [2]:
import asyncio

async def main():
    print('hello')
    await asyncio.sleep(1)
    print('world')

await main()                                      #asyncio.run(main())

hello
world


Note that simply calling a coroutine will not schedule it to be executed:

In [3]:
main()

<coroutine object main at 0x7fd8886c62c0>

To actually run a coroutine, asyncio provides three main mechanisms:
1. The asyncio.run() function to run the top-level entry point “main()” function (see the above example.)
2. Awaiting on a coroutine. The following snippet of code will print “hello” after waiting for 1 second, and then print “world” after waiting for another 2 seconds:

In [5]:
import asyncio
import time

async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    print(f"started at {time.strftime('%X')}")
    
    await say_after(1, 'hello')
    await say_after(2, 'world')
    
    print(f"finished at {time.strftime('%X')}")
    
await main()                  #asyncio.run(main())

started at 11:12:03
hello
world
finished at 11:12:06


3. The asyncio.create_task() function to run coroutines concurrently as asyncio Tasks.

Let’s modify the above example and run two say_after coroutines concurrently:

In [8]:
async def main():
    task1 = asyncio.create_task(say_after(1, 'hello'))
    task2 = asyncio.create_task(say_after(2, 'world'))
    
    print(f'started at {time.strftime("%X")}')
    
    await task1
    await task2
    
    print(f'finished at {time.strftime("%X")}')
    
await main()

started at 11:15:09
hello
world
finished at 11:15:11


Note that expected output now shows that the snippet runs 1 second faster than before

### Awaitables

We say that an object is an awaitable object if it can be used in an await expression. Many asyncio APIs are designed to accept awaitables.

There are three main types of awaitable objects: 
1. coroutines
2. Tasks
3. Futures

#### 1. Coroutines

Python coroutines are awaitables and therefore can be awaited from other coroutines:

In [11]:
import asyncio

async def nested():
    return 45

async def main():
    # Nothing happens if we just call "nested()".
    # A coroutine object is created but not awaited,
    # so it *won't run at all*.
    nested()
    
    # Let's do it differently now and await it:
    print(await nested())  # will print "42".

await main()

45


<ipython-input-11-5b4ab6372072>:10: RuntimeWarning: coroutine 'nested' was never awaited
  nested()


Important In this documentation the term “coroutine” can be used for two closely related concepts:
- a coroutine function: an async def function;
- a coroutine object: an object returned by calling a coroutine function.

asyncio also supports legacy generator-based coroutines.

#### 2. Tasks

Tasks are used to schedule coroutines concurrently.

When a coroutine is wrapped into a Task with functions like asyncio.create_task() the coroutine is automatically scheduled to run soon:

In [16]:
import asyncio

async def nested():
    return 234

async def main():
    task = asyncio.create_task(nested())
    
    result = await task
    
    print(result)
    
await main()

234


#### 3. Futures

A Future is a special low-level awaitable object that represents an eventual result of an asynchronous operation.

When a Future object is awaited it means that the coroutine will wait until the Future is resolved in some other place.

Future objects in asyncio are needed to allow callback-based code to be used with async/await.

Normally there is no need to create Future objects at the application level code.

Future objects, sometimes exposed by libraries and some asyncio APIs, can be awaited:

In [17]:
async def main():
    await function_that_returns_a_future_object()

    # this is also valid:
    await asyncio.gather(
        function_that_returns_a_future_object(),
        some_python_coroutine()
    )

A good example of a low-level function that returns a Future object is loop.run_in_executor().

### Running an asyncio Program

asyncio.run(coro, *, debug=False)

Execute the coroutine coro and return the result.

This function runs the passed coroutine, taking care of managing the asyncio event loop, finalizing asynchronous generators, and closing the threadpool.

This function cannot be called when another asyncio event loop is running in the same thread.

If debug is True, the event loop will be run in debug mode.

This function always creates a new event loop and closes it at the end. It should be used as a main entry point for asyncio programs, and should ideally only be called once.

### Creating Tasks

asyncio.create_task(coro, *, name=None)

Wrap the coro coroutine into a Task and schedule its execution. Return the Task object.

If name is not None, it is set as the name of the task using Task.set_name().

The task is executed in the loop returned by get_running_loop(), RuntimeError is raised if there is no running loop in current thread.

This function has been added in Python 3.7. Prior to Python 3.7, the low-level asyncio.ensure_future() function can be used instead:

In [21]:
async def coro():
    ...

# In Python 3.7+
task = asyncio.create_task(coro())
...

# This works in all Python versions but is less readable
task = asyncio.ensure_future(coro())
...

Ellipsis

### Sleeping

coroutine asyncio.sleep(delay, result=None, *, loop=None)

Block for delay seconds.

If result is provided, it is returned to the caller when the coroutine completes.

sleep() always suspends the current task, allowing other tasks to run.

Deprecated since version 3.8, will be removed in version 3.10: The loop parameter.

Example of coroutine displaying the current date every second for 5 seconds:

In [23]:
import asyncio
import datetime

async def display_date():
    loop = asyncio.get_running_loop()
    end_time = loop.time() + 5.0
    while True:
        print(datetime.datetime.now())
        if (loop.time() + 1.0) >= end_time:
            break
        await asyncio.sleep(1)

await display_date()

2021-02-24 12:25:10.621707
2021-02-24 12:25:11.622401
2021-02-24 12:25:12.628065
2021-02-24 12:25:13.633620
2021-02-24 12:25:14.639197


### Running Tasks Concurrently¶

awaitable asyncio.gather(*aws, loop=None, return_exceptions=False)

Run awaitable objects in the aws sequence concurrently.

If any awaitable in aws is a coroutine, it is automatically scheduled as a Task.

If all awaitables are completed successfully, the result is an aggregate list of returned values. The order of result values corresponds to the order of awaitables in aws.

If return_exceptions is False (default), the first raised exception is immediately propagated to the task that awaits on gather(). Other awaitables in the aws sequence won’t be cancelled and will continue to run.

If return_exceptions is True, exceptions are treated the same as successful results, and aggregated in the result list.

If gather() is cancelled, all submitted awaitables (that have not completed yet) are also cancelled.

If any Task or Future from the aws sequence is cancelled, it is treated as if it raised CancelledError – the gather() call is not cancelled in this case. This is to prevent the cancellation of one submitted Task/Future to cause other Tasks/Futures to be cancelled.

Deprecated since version 3.8, will be removed in version 3.10: The loop parameter.

In [25]:
import asyncio

async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        print(f"Task {name}: Compute factorial({i})...")
        await asyncio.sleep(1)
        f *= 1
    print(f"Task {name}: factorial({number}) = {f}")

    
async def main():
    
    await asyncio.gather(
        factorial('A', 2),
        factorial('B', 3),
        factorial('C', 4)
    )

await main()

Task A: Compute factorial(2)...
Task B: Compute factorial(2)...
Task C: Compute factorial(2)...
Task A: factorial(2) = 1
Task B: Compute factorial(3)...
Task C: Compute factorial(3)...
Task B: factorial(3) = 1
Task C: Compute factorial(4)...
Task C: factorial(4) = 1


### Shielding From Cancellation

awaitable asyncio.shield(aw, *, loop=None)

Protect an awaitable object from being cancelled.

If aw is a coroutine it is automatically scheduled as a Task.

The statement:

res = await shield(something())

is equivalent to:

res = await something()

except that if the coroutine containing it is cancelled, the Task running in something() is not cancelled. From the point of view of something(), the cancellation did not happen. Although its caller is still cancelled, so the “await” expression still raises a CancelledError.

If something() is cancelled by other means (i.e. from within itself) that would also cancel shield().

If it is desired to completely ignore cancellation (not recommended) the shield() function should be combined with a try/except clause, as follows:

In [ ]:
try:
    res = await shield(something())
except CancelledError:
    res = None

### Timeouts

coroutine asyncio.wait_for(aw, timeout, *, loop=None)

Wait for the aw awaitable to complete with a timeout.

If aw is a coroutine it is automatically scheduled as a Task.

timeout can either be None or a float or int number of seconds to wait for. If timeout is None, block until the future completes.

If a timeout occurs, it cancels the task and raises asyncio.TimeoutError.

To avoid the task cancellation, wrap it in shield().

The function will wait until the future is actually cancelled, so the total wait time may exceed the timeout. If an exception happens during cancellation, it is propagated.

If the wait is cancelled, the future aw is also cancelled.

Deprecated since version 3.8, will be removed in version 3.10: The loop parameter.

In [27]:
async def eternity():
    # Sleep for one hour
    await asyncio.sleep(3600)
    print('yay!')

async def main():
    # Wait for at most 1 second
    try:
        await asyncio.wait_for(eternity(), timeout=1.0)
    except asyncio.TimeoutError:
        print('timeout!')

await main()

timeout!
